In [3]:
import os
import sys
import pathlib
import shutil
import math

import tensorflow as tf
from tensorflow import keras
from keras import backend as K
from keras.layers import Input, Conv2D, Dense, GlobalAveragePooling2D, \
                         MaxPooling2D, Dropout, BatchNormalization
from keras.losses import SparseCategoricalCrossentropy
from keras.metrics import SparseCategoricalAccuracy
from keras.regularizers import l2
from tensorflow.keras.optimizers import Adam, SGD

from sklearn.metrics import classification_report, ConfusionMatrixDisplay

import seaborn as sn

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

ImportError: cannot import name 'Adam' from 'keras.optimizers' (/Users/danielnguyen/miniforge3/envs/tk-ML/lib/python3.9/site-packages/keras/optimizers.py)

In [3]:
label_df = pd.read_csv('../gdsc-ai-challenge/trainLabels.csv')

class_names = label_df['label'].unique()
number_of_classes = len(class_names)

In [4]:
def split_dataset(ds, ds_size, train_split=0.8, val_split=0.1, test_split=0.1):
    """Split the dataset into three subsets: train, validation (dev) and test set.

    Returns three tuples, containing each subset with its size.

    Keyword arguments:

    ds -- tf.data.Dataset object

    ds_size -- size of the dataset

    train_split -- percentage to split into train set (default to 0.8)

    val_split -- percentage to split into validation set (default to 0.1)

    test_split -- percentage to split into test set (default to 0.1)
    """
    assert (train_split + test_split + val_split) == 1
    
    train_size = int(train_split * ds_size)
    val_size = int(val_split * ds_size)
    
    train_ds = ds.take(train_size)    
    val_ds = ds.skip(train_size).take(val_size)
    test_ds = ds.skip(train_size).skip(val_size)
    
    return (train_ds, train_size), (val_ds, val_size), (test_ds, ds_size - val_size - train_size)

def configure(ds, ds_size, batch_size=32, shuffle=False, augment=False):
    """Configure the given dataset for better performance (by caching, prefetching and then batching the dataset)

    and perform preprocessing to the images in the given dataset.

    Returns the optimized dataset.

    Keyword arguments:

    ds -- tf.data.Dataset object

    ds_size -- size of the dataset

    batch_size -- size of each batch (default to 32)

    shuffle -- whether to shuffle the dataset (default to False)

    augment -- whether to perform data augmentation to the dataset (default to False)
    """

    AUTOTUNE = tf.data.AUTOTUNE
    rescale = keras.layers.Rescaling(1.0/255)
    data_augmentation = keras.Sequential([
        keras.layers.RandomFlip('horizontal'),
        keras.layers.RandomRotation(0.05, fill_mode='nearest')
    ])

    ds = ds.map(lambda x, y: (rescale(x), y),
                num_parallel_calls=AUTOTUNE)

    if shuffle:
        ds = ds.shuffle(buffer_size=int(ds_size * 0.6))
    
    ds = ds.batch(batch_size)

    if augment:
        with tf.device('/cpu:0'):
            #only perform data augmentation on train set
            ds = ds.map(lambda x, y: (data_augmentation(x, training=True), y),
                                    num_parallel_calls=AUTOTUNE)
    ds = ds.prefetch(buffer_size=AUTOTUNE)
    return ds

In [5]:
ds = tf.keras.utils.image_dataset_from_directory(
    '../gdsc-ai-challenge/train',
    color_mode='rgb',
    batch_size=64,
    image_size=(32,32),
    seed=42
)

ds_size = ds.cardinality().numpy()

Found 50000 files belonging to 10 classes.
Metal device set to: Apple M1

systemMemory: 16.00 GB
maxCacheSize: 5.33 GB



2022-03-23 12:41:47.972755: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-03-23 12:41:47.972890: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [6]:
(train_ds, train_size), (val_ds, val_size), (test_ds, test_size) = split_dataset(ds, 
                                                                        ds_size, 
                                                                        train_split=0.7, 
                                                                        val_split=0.2, 
                                                                        test_split=0.1)

train_ds = configure(train_ds, train_size, augment=True, shuffle=True)
val_ds = configure(val_ds, val_size)
test_ds = configure(test_ds, test_size)

In [ ]:
def generate_version(path_to_dir, new_file_name='', new_version=True):
    """Create a new folder contains new_version weights file

    Returns new_version, path to saving folder and path to save new_file_name (preferably .hdf5 files)

    Keyword arguments:

    path_to_dir -- path to directory where new folder should be created

    new_file_name -- name of the desired file (default to '')

    new_version -- version of the saved model and new weights file (default to 1)
    """

    assert(type(new_version) == bool) , "new_version must be integer"

    versions_list = sorted([int(version.removeprefix('version')) for version in os.listdir(path_to_dir) if version != ".DS_Store"])

    if new_version == True:
        new_version = versions_list[-1] + 1 if len(versions_list) > 0 else 1
    else:
        new_version = versions_list[-1] if len(versions_list) > 0 else 1

    new_version_folder = 'version{}'.format(int(new_version))
    new_version_file = new_file_name if len(new_file_name) > 0 else None

    save_path = path = os.path.join(path_to_dir, new_version_folder)
    if new_version_file is not None:
        save_path = os.path.join(save_path, new_version_file)
    if versions_list.count(new_version) == 0:
        os.makedirs(path)

    return new_version, path, save_path

In [ ]:
version = input("""Create new folder?

                (Y/n)    
                """)

new_version, path, weights_save_path = generate_version('../Model/aiseries', 'weights.best.hdf5', version.lower() == 'y')
_, _, report_save_path = generate_version('../TrainingReport', new_version=version.lower() == 'y')

In [ ]:
%%write_and_run {path}/model.py
import tensorflow as tf
from tensorflow import keras
from keras import regularizers
import os

def create_model(path_to_weights='', load_weights=True):
    """Function to create a model

    Returns a compiled and optionally loaded model

    Keyword arguments:

    path_to_weights -- (Optional, only used when load_weights is True) -- Path to weight file (.hdf5 files)

    load_weights -- Whether to load weights or not (default to True)
    """
    if (load_weights):
        assert(path_to_weights is not None and 
           os.path.isfile(path_to_weights)), "path_to_weights must exist and not be empty if load_weights is True, otherwise change load_weights to False"

    model = keras.models.Sequential([
        Input((32,32,3)),
        Dropout(0.2),
        Conv2D(96, (3,3), padding='same',
                            kernel_regularizer=l2(1e-3),
#                             activity_regularizer=l2(1e-3),
                            kernel_initializer='he_normal',
                            activation='relu'),
        BatchNormalization(),
        Conv2D(96, (3,3), padding='same',
                            kernel_regularizer=l2(1e-3),
#                             activity_regularizer=l2(1e-3),
                            kernel_initializer='he_normal',
                            activation='relu'),
        BatchNormalization(),
        Conv2D(96, (3,3), padding='same', strides=(2,2),
                            kernel_regularizer=l2(1e-3),
#                             activity_regularizer=l2(1e-3),
                            kernel_initializer='he_normal',
                            activation='relu'),
        BatchNormalization(),
        Dropout(0.5),

        Conv2D(192, (3,3), padding='same',
                            kernel_regularizer=l2(1e-3),
#                             activity_regularizer=l2(1e-3),
                            kernel_initializer='he_normal',
                            activation='relu'),
        BatchNormalization(),
        Conv2D(192, (3,3), padding='same',
                            kernel_regularizer=l2(1e-3),
#                             activity_regularizer=l2(1e-3),
                            kernel_initializer='he_normal',
                            activation='relu'),
        BatchNormalization(),
        Conv2D(192, (3,3), padding='same', strides=(2,2),
                            kernel_regularizer=l2(1e-3),
#                             activity_regularizer=l2(1e-3),
                            kernel_initializer='he_normal',
                            activation='relu'),
        BatchNormalization(),
        Dropout(0.5),
        
        Conv2D(192, (3,3), padding='same',
                            kernel_regularizer=l2(1e-3),
#                             activity_regularizer=l2(1e-3),
                            kernel_initializer='he_normal',
                            activation='relu'),
        BatchNormalization(),
        Conv2D(192, (1,1), padding='same',
                            kernel_regularizer=l2(1e-3),
#                             activity_regularizer=l2(1e-3),
                            kernel_initializer='he_normal',
                            activation='relu'),
        BatchNormalization(),
        Conv2D(10, (1,1), padding='same',
                            kernel_regularizer=l2(1e-3),
#                             activity_regularizer=l2(1e-3),
                            kernel_initializer='he_normal',
                            activation='relu'),
        BatchNormalization(),
        GlobalAveragePooling2D(),
        
        Dropout(0.5),
        Dense(10, activation='softmax')
    ])

    if load_weights:
        model.load_weights(path_to_weights)

    model.compile(optimizer=Adam(learning_rate=0.001),
                                 loss='sparse_categorical_crossentropy',
                                 metrics=[
                                 SparseCategoricalCrossentropy(name='sparse'),
                                 'accuracy'])

    return model

In [ ]:
model = create_model(load_weights=False)

tf.keras.utils.plot_model(model, os.path.join(path, 'model.png'), show_shapes=True)

model.summary()